<a href="https://colab.research.google.com/github/AlissonGranja977/Proyectofinal/blob/main/Proyecto_redes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Reshape, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Reshape, LeakyReLU
from tensorflow.keras.optimizers import Adam


In [ ]:
# Define the generator
generator = Sequential([
    Dense(256, input_shape=(100,), activation='relu'),
    Dense(512, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(28*28*1, activation='tanh'),
    Reshape((28, 28, 1))
])

In [ ]:
# Define the discriminator
discriminator = Sequential([
    Conv2D(64, (3,3), strides=(2,2), padding='same', input_shape=(28, 28, 1)),
    LeakyReLU(0.2),
    Dropout(0.3),
    Conv2D(128, (3,3), strides=(2,2), padding='same'),
    LeakyReLU(0.2),
    Dropout(0.3),
    Flatten(),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the discriminator
discriminator.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Combine the generator and discriminator to create the GAN
gan = Sequential([generator, discriminator])
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

In [ ]:
# Set the batch size and number of epochs
batch_size = 10
epochs = 10000

In [ ]:
# Freeze the discriminator's weights
discriminator.trainable = False


In [ ]:
# Define the GAN
gan = Sequential([
    generator,
    discriminator
])

In [ ]:
# Compile the GAN
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
# Bring the data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import data from Google Drive
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nibabel as nib
import os
import numpy as np

# Obtener todas las imágenes en la carpeta
folder_path = "/content/drive/MyDrive/BrainTumour/imagesTr"
nii_files = os.listdir(folder_path)

# Crear una lista vacía para guardar las imágenes
images = []

# Iterar sobre cada archivo de la carpeta
for file in nii_files:
    # Si el archivo termina en .nii.gz, cargar la imagen y añadirla a la lista
    if file.endswith(".nii.gz"):
        img_path = os.path.join(folder_path, file)
        img = nib.load(img_path)
        images.append(img)

# Imprimir la longitud de la lista de imágenes para verificar que se han cargado correctamente
print("Número de imágenes cargadas:", len(images))



Número de imágenes cargadas: 484


In [ ]:
import glob
import os
import nibabel as nib
import numpy as np

def normalize_image(img):
    # Obtener los datos de la imagen como un array numpy
    data = img.get_fdata()

    # Calcular la media y la desviación estándar de los datos
    mean_value = np.mean(data)
    std_value = np.std(data)

    # Normalizar la imagen
    normalized_image = (data - mean_value) / std_value

    # Crear una nueva imagen normalizada
    normalized_img = nib.Nifti1Image(normalized_image, img.affine, img.header)

    return normalized_img


# Carpeta con las imágenes originales
folder_path = "/content/drive/MyDrive/BrainTumour/imagesTr"

# Carpeta donde se guardarán las imágenes normalizadas
output_folder_path = "/content/drive/MyDrive/BrainTumour/imagesTr_normalized"

# Crear la carpeta de salida si no existe
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Obtener la lista de archivos .nii.gz en la carpeta
nii_files = glob.glob(os.path.join(folder_path, "*.nii.gz"))

# Iterar sobre cada archivo y normalizar la imagen
for file_path in nii_files:
    # Cargar la imagen
    img = nib.load(file_path)

    # Normalizar la imagen
    normalized_img = normalize_image(img)

    # Guardar la imagen normalizada en la carpeta de salida
    output_file_path = os.path.join(output_folder_path, "normalized_" + os.path.basename(file_path))
    nib.save(normalized_img, output_file_path)

    print("Imagen normalizada guardada en", output_file_path)


Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_001.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_002.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_003.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_004.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_005.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_006.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_007.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/normalized_BRATS_008.nii.gz
Imagen normalizada guardada en /content/drive/MyDrive/BrainTumour/imagesTr_normalized/no

In [ ]:
import glob

folder_path = "/content/drive/MyDrive/BrainTumour/imagesTr_normalized"
nii_files = glob.glob(folder_path + "/*.nii.gz")


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los archivos en conjuntos de entrenamiento (80%), validación (10%) y prueba (10%)
train_files, test_files = train_test_split(nii_files, test_size=0.2, random_state=42)
val_files, test_files = train_test_split(test_files, test_size=0.5, random_state=42)


In [ ]:
import nibabel as nib
import glob
import os
import numpy as np

folder_path = "/content/drive/MyDrive/BrainTumour/imagesTr_normalized"
train_files = glob.glob(os.path.join(folder_path, "normalized_*"))

train_data = []
val_data = []
test_data = []

batch_size = 1

# Dividir los archivos de entrenamiento en lotes y cargar cada lote por separado
for i in range(0, len(train_files), batch_size):
    batch_files = train_files[i:i+batch_size]
    batch_data = []
    for file_path in batch_files:
        img = nib.load(file_path)
        batch_data.append(img.get_fdata())
    train_data.append(np.array(batch_data))

val_files = glob.glob(os.path.join(folder_path, "normalized_*val.nii.gz"))

# Dividir los archivos de validación en lotes y cargar cada lote por separado
for i in range(0, len(val_files), batch_size):
    batch_files = val_files[i:i+batch_size]
    batch_data = []
    for file_path in batch_files:
        img = nib.load(file_path)
        batch_data.append(img.get_fdata())
    val_data.append(np.array(batch_data))

test_files = glob.glob(os.path.join(folder_path, "normalized_*test.nii.gz"))

# Dividir los archivos de prueba en lotes y cargar cada lote por separado
for i in range(0, len(test_files), batch_size):
    batch_files = test_files[i:i+batch_size]
    batch_data = []
    for file_path in batch_files:
        img = nib.load(file_path)
        batch_data.append(img.get_fdata())
    test_data.append(np.array(batch_data))


In [ ]:
import nibabel as nib
import glob
import os
import numpy as np
from scipy import ndimage

folder_path = "/content/drive/MyDrive/BrainTumour/imagesTr_normalized"
train_files = glob.glob(os.path.join(folder_path, "normalized_*"))

img_shape = (128, 128, 128)  # Tamaño de la imagen redimensionada
batch_size = 10

val_files = glob.glob(os.path.join(folder_path, "normalized_*val.nii.gz"))
test_files = glob.glob(os.path.join(folder_path, "normalized_*test.nii.gz"))
def data_generator(files):
    while True:
        batch_files = np.random.choice(files, batch_size)
        batch_data = []
        for file_path in batch_files:
            img = nib.load(file_path)
            img_data = img.get_fdata()
            img_data_resized = ndimage.zoom(img_data, (img_shape[0]/img_data.shape[0], img_shape[1]/img_data.shape[1], img_shape[2]/img_data.shape[2]), order=1)  # Redimensionar la imagen
            batch_data.append(img_data_resized)
        yield np.array(batch_data)

train_data_generator = data_generator(train_files)
val_data_generator = data_generator(val_files)
test_data_generator = data_generator(test_files)


In [ ]:
# Set the batch size and number of epochs
batch_size = 10
epochs = 100

batch_size = 10
epochs = 10000



In [ ]:
# Train the GAN
for epoch in range(epochs):
    # Generate random noise
    noise = np.random.normal(0, 1, size=(batch_size, 100))
    
    # Generate fake images
    fake_images = generator.predict(noise)
    
    # Concatenate real and fake images
    x = np.concatenate([train_data, fake_images], axis=0)
    
    # Generate labels for real and fake images
    y_real = np.ones((train_data.shape[0], 1))
    y_fake = np.zeros((batch_size, 1))
    y = np.concatenate([y_real, y_fake], axis=0)
    
    # Train the discriminator on real images
    d_loss_real = discriminator.train_on_batch(train_data, y_real)
    
    # Train the discriminator on fake images
    d_loss_fake = discriminator.train_on_batch(fake_images, y_fake)
    
    # Calculate the discriminator loss
    discriminator_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Generate new random noise
    noise = np.random.normal(0, 1, size=(batch_size, 100))
    
    # Train the GAN on the random noise
    gan_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    

    # Print the loss for each epoch
    if epoch % 10 == 0:
      print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss.history['loss'][-1]}, GAN Loss: {gan_loss.history['loss'][-1]}")


NameError: ignored

In [ ]:
# Set the batch size and number of epochs
batch_size = 5
epochs = 8


In [ ]:
# Define a function to generate random noise
def generate_noise(batch_size, noise_size):
    return tf.random.normal((batch_size, noise_size))

In [ ]:
# Train the GAN
for epoch in range(epochs):
    # Generate random noise
    noise = generate_noise(batch_size, 100)
    
    # Generate fake images
    fake_images = generator.predict(noise)
    
    # Concatenate real and fake images
    x = tf.concat([x_train, fake_images], axis=0)
    
    # Generate labels for real and fake images
    y_real = tf.ones((x_train.shape[0], 1))
    y_fake = tf.zeros((x_train.shape[0], 1))
    y = tf.concat([y_real, y_fake], axis=0)
    
    # Train the discriminator on real images
    d_loss_real = discriminator.train_on_batch(x_train, tf.ones((batch_size, 1)))
    
    # Generate new random noise
    noise = generate_noise(batch_size, 100)

    # Generate fake images
    fake_images = generator.predict(noise)

    # Train the discriminator on fake images
    d_loss_fake = discriminator.train_on_batch(fake_images, tf.zeros((batch_size, 1)))

    # Calculate the discriminator loss
    discriminator_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Generate new random noise
    noise = generate_noise(batch_size, 100)

    # Train the GAN on the random noise
    gan_loss = gan.train_on_batch(noise, tf.ones((batch_size, 1)))

    # Print the loss for each epoch
    if epoch % 100 == 0:
      print(f"Epoch: {epoch}, Discriminator Loss: {discriminator_loss}, GAN Loss: {gan_loss}")

NameError: ignored